In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

2022-03-15 23:01:14.330923: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-15 23:01:14.331304: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-15 23:01:14.333854: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [2]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [3]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

# model.fit(
#     ds_train,
#     epochs=6,
#     validation_data=ds_test,
# )

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [13]:
totalParams

101770.0

In [29]:
import node

input = tf.keras.Input(shape=(28, 28))
flatten = tf.keras.layers.Flatten()
flat_input = flatten(input)

nb_layers = 8
layer1 = node.DenseLayer(nb_layers, activation='relu')
layer2 = node.DenseLayer(nb_layers, activation='relu')
layer3 = node.DenseLayer(nb_layers, activation='relu')
layer4 = node.DenseLayer(nb_layers, activation='relu')

last_active_layer = layer1.create_node(dataset_size=(28, 28))(flat_input)
last_active_layer = layer2.create_node(dataset_size=(28, 28))(last_active_layer)
last_active_layer = layer3.create_node(dataset_size=(28, 28))(last_active_layer)
last_active_layer = layer4.create_node(dataset_size=(28, 28))(last_active_layer)

output = tf.keras.layers.Dense(units=10, activation='softmax')(last_active_layer)
model2 = tf.keras.Model(input, output)

model2.compile(
    optimizer=tf.keras.optimizers.Adam(0.001), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

history = model2.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)
      
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(input_shape=(28, 28)),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dense(10)
# ])
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(0.001),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
# )

# model.fit(
#     ds_train,
#     epochs=6,
#     validation_data=ds_test,
# )

Epoch 1/6
469/469 [==============================] - 7s 13ms/step - loss: 1.5902 - sparse_categorical_accuracy: 0.4707 - val_loss: 0.5601 - val_sparse_categorical_accuracy: 0.8397
Epoch 2/6
469/469 [==============================] - 4s 8ms/step - loss: 0.5408 - sparse_categorical_accuracy: 0.8418 - val_loss: 0.4506 - val_sparse_categorical_accuracy: 0.8684
Epoch 3/6
469/469 [==============================] - 4s 8ms/step - loss: 0.4538 - sparse_categorical_accuracy: 0.8673 - val_loss: 0.3787 - val_sparse_categorical_accuracy: 0.8892
Epoch 4/6
469/469 [==============================] - 3s 7ms/step - loss: 0.3821 - sparse_categorical_accuracy: 0.8904 - val_loss: 0.3418 - val_sparse_categorical_accuracy: 0.9028
Epoch 5/6
469/469 [==============================] - 3s 6ms/step - loss: 0.3372 - sparse_categorical_accuracy: 0.9027 - val_loss: 0.3169 - val_sparse_categorical_accuracy: 0.9089
Epoch 6/6
469/469 [==============================] - 3s 7ms/step - loss: 0.3175 - sparse_categorical_acc

In [36]:
ds_info

tfds.core.DatasetInfo(
    name='mnist',
    version=1.0.0,
    description='The MNIST database of handwritten digits.',
    urls=['https://storage.googleapis.com/cvdf-datasets/mnist/'],
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)